__Maze Example:__


Consider a rectangle grid of rooms, where each room may or may not have doors on the North, South, East, and West sides.

How do you find your way out of a maze? Here is one possible "algorithm" for finding the answer:

For every door in the current room, if the door leads to the exit, take that door.

The "trick" here is of course, how do we know if the door leads to a room that leads to the exit? The answer is we don't but we can let the computer figure it out for us.

What is the recursive part about the above algorithm? Its the "door leads out of the maze". How do we know if a door leads out of the maze? We know because inside the next room (going through the door), we ask the same question, how do we get out of the maze?

What happens is the computer "remembers" all the "what ifs". What if I take the first door, what if I take the second door, what if I take the next door, etc. And for every possible door you can move through, the computer remembers those what ifs, and for every door after that, and after that, etc, until the end is found.

Here is a close to actual code implementation.

In [1]:
import math

def can_escape(maze, i=0, j=0):
    # (i, j) is the starting position
    # maze[x][y] = 0 <=> (x, y) cell is empty
    # maze[x][y] = 1 <=> (x, y) cell contains a wall
    n = len(maze)
    m = len(maze[0])
    # print(i,j)
    if i == n - 1 and j == m - 1:
        return True
    # mark passed maze as 1, so we won't visit it again
    maze[i][j] = 1
    result = False
    for a, b in [(i - 1, j), (i, j - 1), (i + 1, j), (i, j + 1)]:
        # when there is a way out, continue to next
        if 0 <= a < n and 0 <= b < m and maze[a][b] == 0:
            # print('go to ', a,b, result)
            result = result or can_escape(maze, a, b)
    maze[i][j] = 0
    return result

def fastest_escape_length(maze, i=0, j=0, escape_length = 1, min_length = math.inf):
    """
    Write a function fastest_escape_length(maze) that takes a maze (as above) as input 
    and returns the length of the shortest path from the upper left corner to the lower right corner. 
    The length of a path is the number of cells it passes through. 
    If there is no path, the function should return math.inf.
    """
    n = len(maze)
    m = len(maze[0])
    # print(i,j)
    if i == n - 1 and j == m - 1:
        # print('---', escape_length, min_length)
        if escape_length < min_length:
            min_length = escape_length
        return min_length
    maze[i][j] = 1
    escape_length += 1
    # result = False
    for a, b in [(i - 1, j), (i, j - 1), (i + 1, j), (i, j + 1)]:
        # print(a,b)
        if 0 <= a < n and 0 <= b < m and maze[a][b] == 0:
            # print('in', (a,b))
            if escape_length < min_length: 
                min_length = fastest_escape_length(maze, a, b, escape_length, min_length)

    maze[i][j] = 0
    # print(escape_length)
    return min_length

def fastest_escapes(maze, i=0, j=0, current_path = '', escape_path= None):
    """
    Write a function fastest_escapes(maze) that takes a maze and 
    returns a list of all shortest paths from the upper left corner to the lower right corner. 
    Each path should be represented as a list of cells along the path where a cell is a tuple of the coordinates. 
    In the example above, the output should be [ [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)], [(0, 0), (0, 1), (1, 1), (1, 2), (2, 2)] ]
    """
    n = len(maze)
    m = len(maze[0])

    if i == 0 and j == 0:
        escape_path = []

    # print(i,j)
    current_path = current_path + str(i)+','+str(j) +';'

    if i == n - 1 and j == m - 1:
        # print('escaped_path', current_path)

        current_path = current_path.rstrip(';')

        arr = [tuple(map(int, x.split(','))) for x in current_path.split(';')]

        if len(escape_path) == 0:
            escape_path.append(arr)
        elif len(arr) < len(escape_path[0]):
            escape_path[0] = arr
            # remove all the others which equal to the previous escape length
            while len(escape_path) > 1:
                escape_path.pop()
        elif len(arr) == len(escape_path[0]) :
            escape_path.append(arr)
        return escape_path
        
    maze[i][j] = 1

    # result = False
    for a, b in [(i - 1, j), (i, j - 1), (i + 1, j), (i, j + 1)]:
        # print(a,b)
        if 0 <= a < n and 0 <= b < m and maze[a][b] == 0:
            if len(escape_path)==0 or len(current_path)/4 < len(escape_path[0]):
                # if(i == 0 and j == 0) and len(escape_path) > 1:
                #     escape_path = []
                fastest_escapes(maze, a, b, current_path, escape_path)

    # print('current_path', current_path)

    maze[i][j] = 0
    # print(escape_length)
    return escape_path

def weighted_escape_length(maze, w, i=0, j=0, escape_length = 0, min_length = math.inf):
    """
    Write a function weighted_escape_length(maze, w) that takes a maze and a nonegative integer w and 
    returns the length of the shortest path from the upper left corner to the lower right corner if it is allowed to pass through the walls, 
    but each wall is considered equivalent to w empty cells.
    """
    n = len(maze)
    m = len(maze[0])

    if maze[i][j] == 0:
        escape_length += 1
    elif maze[i][j] == 1:
        escape_length += w * maze[i][j]
    # print(i,j)
    if i == n - 1 and j == m - 1:
        # print('---', escape_length, min_length)
        if escape_length < min_length:
            min_length = escape_length
        return min_length

    maze[i][j] += 2
    
    # result = False
    for a, b in [(i - 1, j), (i, j - 1), (i + 1, j), (i, j + 1)]:
        # print(a,b)
        if 0 <= a < n and 0 <= b < m and maze[a][b] < 2:
            # print('in', (a,b))
            if escape_length < min_length: 
                min_length = weighted_escape_length(maze, w, a, b, escape_length, min_length)

    maze[i][j] -= 2
    # print(escape_length)
    return min_length

def weighted_escapes(maze, w, i=0, j=0, escape_length = 0, min_length = None, current_path = '', escape_path= None):
    """
    Write a function weighted_escapes(maze, w) that returns a list of shortest paths in the sense of Part weighted_escape_length. 
    For example, for the sample input of Part 3, the output should be [ [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)] ].

    The time limit for each test will be one second.
    """
    n = len(maze)
    m = len(maze[0])

    if i == 0 and j == 0:
        escape_path = []
        min_length = [math.inf]

    if maze[i][j] == 0:
        escape_length += 1
    elif maze[i][j] == 1:
        escape_length += w * maze[i][j]

    # print(i,j)
    current_path = current_path + str(i)+','+str(j) +';'

    if i == n - 1 and j == m - 1:
        # print('escaped_path', current_path)

        current_path = current_path.rstrip(';')

        arr = [tuple(map(int, x.split(','))) for x in current_path.split(';')]
        # print(escape_length, min_length)
        if len(escape_path) == 0: # can use min_length==math.info too
            min_length[0] = escape_length
            escape_path.append(arr)
        elif escape_length < min_length[0]:
            min_length[0] = escape_length
            escape_path[0] = arr
            # remove all the others which equal to the previous escape length
            while len(escape_path) > 1:
                escape_path.pop()
        elif escape_length == min_length[0] :
            escape_path.append(arr)
        return escape_path
        
    maze[i][j] += 2

    # result = False
    for a, b in [(i - 1, j), (i, j - 1), (i + 1, j), (i, j + 1)]:
        # print(a,b)
        if 0 <= a < n and 0 <= b < m and maze[a][b] < 2:
            if escape_length < min_length[0]:
                # if(i == 0 and j == 0) and len(escape_path) > 1:
                #     escape_path = []
                weighted_escapes(maze, w, a, b, escape_length, min_length, current_path, escape_path)

    # print('current_path', current_path)

    maze[i][j] -= 2
    # print(escape_length)
    return escape_path


# some test code
if __name__ == "__main__":

    test1 = [
        [0,1,1,1,0],
        [0,0,0,0,0],
        [0,1,0,1,0],
        [0,0,0,0,0],
    ]

    # test1 = [
    #     [0,0,0],
    #     [0,1,0],
    #     [0,0,0],
    # ]

    # print(weighted_escapes(test1, 0.9))

    test_a = [
        [0, 0, 0],
        [1, 1, 0],
        [1, 1, 0]
    ]
    # # should print 5
    # print(fastest_escape_length(test_a))
    # # should print 2
    # print(weighted_escape_length(test_a, 0))
    test_b = [
        [0, 0, 0],
        [1, 1, 1],
        [0, 0, 0]
    ]
    # # should print inf
    # print(fastest_escape_length(test_b))
    # # should print 5
    # print(weighted_escape_length(test_b, 1))
    # # should print 6
    # print(weighted_escape_length(test_b, 2))

    # # should print [[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]]
    # print(fastest_escapes(test_a))
    # # should print []
    # print(fastest_escapes(test_b))
    # # should print [5, 5, 5, 5, 5, 5]
    # print(list(map(len, fastest_escapes([[0 for _ in range(3)] for _ in range(3)]))))

    # should print [[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2)]]
    print(weighted_escapes(test_b, 0))
    # should print [[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)], [(0, 0), (0, 1), (1, 1), (2, 1), (2, 2)], [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]]
    # the order of the paths within the list might be different
    print(weighted_escapes(test_b, 2))

[[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2)]]
[[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)], [(0, 0), (0, 1), (1, 1), (2, 1), (2, 2)], [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]]


In [ ]:
function carve_passages_from(cx, cy, grid){
  directions = [N, S, E, W].sort_by{rand}

  directions.each do |direction|{
    nx, ny = cx + DX[direction], cy + DY[direction]

    if ny.between?(0, grid.length-1) && nx.between?(0, grid[ny].length-1) && grid[ny][nx] == 0{
      grid[cy][cx] |= direction
      grid[ny][nx] |= OPPOSITE[direction]
      carve_passages_from(nx, ny, grid)
    }
  }
}